## 目录

- [Packages](#0)
- [1 开始进行J × K 折交叉验证](#1)
- [2 获取特征重要性](#2)
- [3 对重要特征进行分析](#3)
    - [3.1行人年龄段](#3-1)
    - [3.2事故发生时间](#3-2)
    - [3.3道路类型](#3-3)
    - [3.4道路最高允许车速](#3-4)

<a name='0'></a>
# Packages
Run the following cell to load the packages.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
data = pd.read_excel("数据集.xlsx")   

<img src="X&Y.png" style="width:600px;height:800;">

In [ ]:
data.head()

In [ ]:
data.info()

<a name='1'></a>
# 1 开始进行J × K 折交叉验证

In [ ]:
rf = RandomForestClassifier()

all_scores：每一次打乱数据集顺序的交叉验证的AUC分数，每次返回5个分数。共有50个分数。
                           
all_means：每一次打乱数据集顺序的交叉验证的5次AUC得分的均值，每次返回1个分数。共有10个分数。

In [ ]:
all_scores = []
all_means = []

for i in range(10):
    data = shuffle(data)    
    scores = cross_val_score(rf, X = data.loc[:,'事故发生时间':'道路类型'], y = data['行人事故后果'], cv=5, scoring='roc_auc')
    mean = scores.mean()
    all_scores.append(scores)
    all_means.append(mean)

In [ ]:
all_scores

In [ ]:
all_means

In [ ]:
sum(all_means) / 10    # 所有AUC的均值，即all_means的均值

<a name='2'></a>
# 2 获取特征重要性

In [ ]:
X = data.loc[:, '事故发生时间':'道路类型']
y = data['行人事故后果']
X.shape, y.shape

In [ ]:
rf = RandomForestClassifier()
rf.fit(X, y)
y_pred = rf.predict(X) 

In [ ]:
# 计算特征重要性
importances = rf.feature_importances_

# 对特征重要性进行排序
indices = np.argsort(importances)[::-1]

# 获取特征名字
names = [X.columns.values[i] for i in indices]

#对特征重要性进行排序
good = sorted(importances, reverse=True)
good = [i * 100 for i in good]    #化为百分号表示

In [ ]:
c = {"特征" : names,
   "特征重要度（%）" : good} 
fea_imp = pd.DataFrame(c) 
fea_imp = fea_imp.round(2)    # 保留小数点后两位
fea_imp

In [ ]:
plt.rcParams['font.sans-serif'] = ['SimHei']   #用来正常显示中文标签

fig = plt.figure(figsize = (16,8))

plt.barh(fea_imp["特征"], fea_imp["特征重要度（%）"],height = 0.5, color='r')  

#显示条形图上的数字
for a,b in zip(list(fea_imp["特征"]),list(fea_imp["特征重要度（%）"])):   
    plt.text(b+0.1, a,'%.2f'%b, ha = 'left',va = 'center',fontsize=20)
 
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlim(0,12)
plt.xlabel('特征重要度（%）',fontsize=20)
plt.ylabel('特征',fontsize=20)
plt.show()

<a name='3'></a>
# 3 对重要特征进行分析

下面所有表格的行人事故后果属性表示的含义是：行人发生死亡或重伤的占比

<a name='3-1'></a>
3.1行人年龄段

In [ ]:
map = {'行人年龄段':data["行人年龄段"].value_counts().index,'次数':data["行人年龄段"].value_counts().values}
df = pd.DataFrame(map)
c = data[["行人年龄段", "行人事故后果"]].groupby(['行人年龄段'], as_index=True).mean().sort_values(by='行人事故后果', ascending=False)
d31 = pd.merge(c, df,on = '行人年龄段')
d31

<a name='3-2'></a>
3.2事故发生时间

In [ ]:
map = {'事故发生时间':data["事故发生时间"].value_counts().index,'次数':data["事故发生时间"].value_counts().values}
df = pd.DataFrame(map)
c = data[["事故发生时间", "行人事故后果"]].groupby(['事故发生时间'], as_index=True).mean().sort_values(by='行人事故后果', ascending=False)
d32 = pd.merge(c, df,on = '事故发生时间')
d32

<a name='3-3'></a>
3.3道路类型

In [ ]:
map = {'道路类型':data["道路类型"].value_counts().index,'次数':data["道路类型"].value_counts().values}
df = pd.DataFrame(map)
c = data[["道路类型", "行人事故后果"]].groupby(['道路类型'], as_index=True).mean().sort_values(by='行人事故后果', ascending=False)
d33 = pd.merge(c, df,on = '道路类型')
d33

<a name='3-4'></a>
3.4道路最高允许车速

In [ ]:
map = {'道路最高允许车速':data["道路最高允许车速"].value_counts().index,'次数':data["道路最高允许车速"].value_counts().values}
df = pd.DataFrame(map)
c = data[["道路最高允许车速", "行人事故后果"]].groupby(['道路最高允许车速'], as_index=True).mean().sort_values(by='行人事故后果', ascending=False)
d34 = pd.merge(c, df,on = '道路最高允许车速')
d34